In [ ]:
# Define BBOX (or Poly), URL A & B
# Reproject BBOX to NAIP crs
# Clip NAIP to BBOX
# Resample NAIPs to same resolution
# Detect Changes btwn NAIPs

In [ ]:
import os
os.chdir('..')
os.system('pwd')

import rioxarray
import shapely

from shapely.geometry import box
from shapely.ops import transform
from pyproj import Transformer

from pathlib import Path
import geoai
from geoai.change_detection import ChangeDetection

import geopandas as gpd

/Users/adamswietek/Documents/PostDoc/HiddenHousing


In [ ]:
''' define bbox, naip url A & B '''
from src.constants import LARGE_BBOX

bbox_wgs84 = LARGE_BBOX
naip_a = "data/01_raw/naip/m_3411861_se_11_1_20140513.tif"
naip_b = "data/01_raw/naip/m_3411861_se_11_060_20220511.tif"

In [ ]:
'''reproject'''

#read, reproject, and save
a_clip = rioxarray.open_rasterio(naip_a)
b_clip = rioxarray.open_rasterio(naip_b)

a_aligned_path = "data/01_raw/naip/m_3411861_se_11_1_20140513_reproj.tif"
a_aligned = a_clip.rio.reproject_match(b_clip)
a_aligned.rio.to_raster(a_aligned_path, tiled=True, compress="LZW")

In [ ]:
import os
from tqdm import tqdm
# Set up paths
out_folder = "data/02_interm_tiles"
Path(out_folder).mkdir(exist_ok=True)


tile_names = list(os.listdir('data/02_interm_tiles/A'))

def init_detector():
    # Make sure model directory exists
    Path("~/.cache/torch/hub/checkpoints/").expanduser().mkdir(parents=True, exist_ok=True)

    # Initialize change detection
    detector = ChangeDetection(sam_model_type="vit_h")

    # Configure parameters (following the torchange example)
    detector.set_hyperparameters(
        change_confidence_threshold=145,
        use_normalized_feature=True,
        bitemporal_match=True,
    )

    detector.set_mask_generator_params(
        points_per_side=32,
        stability_score_thresh=0.95,
    )

    print("Change detection system initialized!")
    return detector

def process_tile(nm):
    tile_A_path = f'data/02_interm_tiles/A/{nm}'
    tile_B_path = f'data/02_interm_tiles/B/{nm}'

    # Run change detection
    results = detector.detect_changes(
        tile_A_path,
        tile_B_path,
        output_path=f"{out_folder}/binary_mask_{nm}",
        export_probability=True,
        probability_output_path=f"{out_folder}/probability_mask_{nm}",
        export_instance_masks=True,
        instance_masks_output_path=f"{out_folder}/instance_masks_{nm}",
        return_detailed_results=True,
        return_results=False,
    )

    print(f"Change detection completed!")
    print(f"Total instances detected: {results['summary']['total_masks']}")
    print(f"Image size: {results['summary']['original_shape']}")
    
    if results and 'summary' in results:
        return results['summary']
    return None
    


In [19]:
tile_names = list(os.listdir('data/02_interm_tiles/A'))
detector = init_detector()

for nm in tile_names[2:4]:
    process_tile(nm)
    


Change detection system initialized!
Saved instance segmentation mask with 66 instances to data/02_interm_tiles/instance_masks_naip_r1_c0.tif
Saved instance scores mask with 66 instances to data/02_interm_tiles/instance_masks_naip_r1_c0_scores.tif
Change detection completed!
Total instances detected: 66
Image size: (1024, 1024)


ERROR 1: PROJ: internal_proj_identify: /Users/adamswietek/opt/anaconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


Saved instance segmentation mask with 66 instances to data/02_interm_tiles/instance_masks_naip_r6_c9.tif
Saved instance scores mask with 66 instances to data/02_interm_tiles/instance_masks_naip_r6_c9_scores.tif
Change detection completed!
Total instances detected: 66
Image size: (1024, 1024)


ERROR 1: PROJ: internal_proj_identify: /Users/adamswietek/opt/anaconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


In [ ]:
'''MERGE SCORE MASKS TILES INTO A SINGLE GEOTIFF'''

import sys
from pathlib import Path
import rasterio
from rasterio.merge import merge

import rasterio
from rasterio.merge import merge
import glob

def merge_masked():
    # Define the directory containing your raster files
    for score_type in ['binary','probability','instance']:
        raster_dir = "data/02_interm_tiles/"
        output_path = f"data/03_processed/{score_type}_mask.tif"

        # Find all raster files (e.g., .tif) in the directory
        search_criteria = raster_dir + f"*{score_type}*.tif"
        raster_files = glob.glob(search_criteria)

        # Open all raster files
        src_files_to_mosaic = []
        for fp in raster_files:
            src = rasterio.open(fp)
            src_files_to_mosaic.append(src)

        # Merge the rasters
        mosaic, out_transform = merge(src_files_to_mosaic)

        # Get the metadata from one of the source rasters and update it for the output
        out_meta = src_files_to_mosaic[0].meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_transform,
            "crs": src_files_to_mosaic[0].crs
        })

        # Write the merged raster to a new file
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(mosaic)

        # Close the source files
        for src in src_files_to_mosaic:
            src.close()

        print(f"Merged raster saved to: {output_path}")



In [ ]:
ref = rioxarray.open_rasterio(naip_a)
bbox_proj = gpd.GeoSeries([box(*bbox_wgs84)], crs="EPSG:4326").to_crs(ref.rio.crs).total_bounds

# Clip both to bbox
clips = {}
for url in (naip_a, naip_b):
    clipped = rioxarray.open_rasterio(url).rio.clip_box(*bbox_proj)
    base_path = url.split('/')[-1]
    clipped_path = f"data/02_interm/{base_path}"
    clipped.rio.to_raster(clipped_path, tiled=True, compress="LZW")
    clips[url] = clipped_path

# Align A to B’s grid for change detection
a_clip = rioxarray.open_rasterio(clips[naip_a])
b_clip = rioxarray.open_rasterio(clips[naip_b])
a_aligned = a_clip.rio.reproject_match(b_clip)
a_aligned.rio.to_raster(clips[naip_a], tiled=True, compress="LZW")



In [ ]:

#  Check if CUDA is available
device = geoai.get_device()
print(f"Using device: {device}")

# Set up paths
out_folder = "data/03_processed/tiles"
Path(out_folder).mkdir(exist_ok=True)

print(f"Working directory: {out_folder}")

In [ ]:

# Make sure model directory exists
Path("~/.cache/torch/hub/checkpoints/").expanduser().mkdir(parents=True, exist_ok=True)

# Initialize change detection
detector = ChangeDetection(sam_model_type="vit_h")

# Configure parameters (following the torchange example)
detector.set_hyperparameters(
    change_confidence_threshold=145,
    use_normalized_feature=True,
    bitemporal_match=True,
)

detector.set_mask_generator_params(
    points_per_side=32,
    stability_score_thresh=0.75,
)

print("Change detection system initialized!")

In [ ]:

# Run change detection
results = detector.detect_changes(
    clips[naip_a],
    clips[naip_b],
    output_path=f"{out_folder}/binary_mask.tif",
    export_probability=True,
    probability_output_path=f"{out_folder}/probability_mask.tif",
    export_instance_masks=True,
    instance_masks_output_path=f"{out_folder}/instance_masks.tif",
    return_detailed_results=True,
    return_results=False,
)

print(f"Change detection completed!")
print(f"Total instances detected: {results['summary']['total_masks']}")
print(f"Image size: {results['summary']['original_shape']}")

In [ ]:
# probability visualization
detector.visualize_results(
    clips[naip_a],
    clips[naip_b],
    f"{out_folder}/binary_mask.tif",
    f"{out_folder}/probability_mask.tif",
)